$\textbf{Empirical Exercise-Week 6}$  Page 402, Problem 8.1
Heteroskedasticity and Labor Market Discrimination 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.covariance import MinCovDet
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import t, f, chi2
%matplotlib inline

In [2]:
Tab1 = pd.read_csv('cps5.csv')

In [3]:
Tab1.head(1)

,age,asian,black,divorced,educ,exper,faminc,female,hrswork,insure,...,metro,midwest,nchild,northeast,single,south,union,wage,west,white
0,45,0,0,0,13,26,39200,1,38,1,...,0,0,0,1,0,0,1,14.38,0,1


In [4]:
wage = np.array(Tab1.iloc[:,20]); educ = np.array(Tab1.iloc[:,4]);
exper = np.array(Tab1.iloc[:,5]); female = np.array(Tab1.iloc[:,7]);
aframer = np.array(Tab1.iloc[:,2]); metro = np.array(Tab1.iloc[:,13]);
south = np.array(Tab1.iloc[:,18]); midwest = np.array(Tab1.iloc[:,14]);
west = np.array(Tab1.iloc[:,21]);

In [5]:
y = np.log(wage)
x = np.vstack([np.ones(len(y)), educ, exper, (exper**2), female, aframer, metro, south, midwest, west]).T

In [6]:
results1 = sm.OLS(y,x).fit();
[results1.params, results1.tvalues]

[array([ 1.20138202e+00,  1.01229613e-01,  2.96216959e-02, -4.45780477e-04,
        -1.65501980e-01, -1.11525250e-01,  1.19020410e-01, -4.57554333e-02,
        -6.39432877e-02, -6.58910218e-03]),
 array([ 37.40888367,  57.57373339,  22.77992395, -16.91478706,
        -17.36799457,  -6.58261129,   9.67114697,  -3.37396013,
         -4.53380516,  -0.45750133])]

You see that the coefficient for female = 1 has a depressive effect on wages.  Given equal education, experience, location, women earn less in the United States.
Lets do a Goldfeld-Quandt test on equalty of variances for the residuals of wages for men and women in the US workforce. 

In [7]:
ehat = results1.resid
ehat_women = ehat[female==1]
ehat_men = ehat[female==0]

In [8]:
Var_women = ehat_women.var();  Var_men = ehat_men.var()
[Var_women, Var_men]

[0.2112234206666959, 0.22147928391399793]

In [9]:
dgf_women = len(ehat_women)-10; dgf_men = len(ehat_men)-10;
F_stat =  (Var_women/dgf_women)/(Var_men/dgf_men)
F_stat

1.1828861974821618

In [10]:
Fprob = f.cdf(F_stat, dgf_women, dgf_men)
pval = 1 - Fprob
pval

2.410876831504538e-09

We can reject the hypothesis of equaltiy of variances between the men and women wage-equation residuals.

New we do the N $R^2$ test for Heteroskeasticity.  We regress the squared residuals on three variables, on Metro, Female, Black and their cross terms. If the null is true, no heteroskedascity, the term N $R^2$ has a $\chi^{2}$ distribution with K*(K+3)/2 degress of freedom

In [11]:
yy = ehat**2
x1 = np.ones(len(yy))

In [12]:
xfa = female * aframer
xfm = female * metro
xma = metro * aframer

In [13]:
xx = np.vstack([x1, female, aframer, metro, xfa, xfm, xma]).T

In [14]:
results2 = sm.OLS(yy,xx).fit()

In [15]:
Chistat = results2.rsquared * len(ehat)
dgf2 = 7*10/2
Chistat

32.29820661387902

In [16]:
Chip = chi2.cdf(Chistat, dgf2)
pval2 = 1 - Chip
pval2

0.5992168652863528

So we caonnot reject homoskedasticity with these variables in this test.  Let repeat it with all of the explanatory variables, as your book suggests


In [17]:
results3 = sm.OLS(yy,x).fit()
dgf4 =  10 * 13/2
Chistat =  len(yy)*results3.rsquared
Chip = chi2.cdf(Chistat, dgf4)
pval3 = 1-Chip
pval3

0.0004718645891164108

We can now reject homoskecasticty when we use all of the regressors.
White test is similar, with quadratic expansion on the variables. 

In [18]:
xxx= np.vstack([np.ones(len(yy)), educ,  educ**2, exper, exper**2, educ*exper, \
female, aframer, metro, south, midwest, west, female**aframer, metro**aframer, \
metro*south, metro*female]).T
results4 = sm.OLS(yy,xxx).fit()
dgf5 = 15
White = len(yy) * results4.rsquared

In [19]:
White_test = chi2.cdf(White,dgf5)
pval4 = 1 - White_test
pval4

0.0

We can reject homoskedasticity with the White test

In [20]:
np.set_printoptions(precision=4,suppress=True,linewidth=100)
print(results1.cov_HC0)

[[ 0.0011 -0.0001 -0.      0.     -0.     -0.     -0.0001 -0.0001 -0.0001 -0.0001]
 [-0.0001  0.     -0.      0.     -0.      0.     -0.      0.      0.      0.    ]
 [-0.     -0.      0.     -0.      0.      0.     -0.     -0.     -0.     -0.    ]
 [ 0.      0.     -0.      0.     -0.      0.      0.      0.      0.      0.    ]
 [-0.     -0.      0.     -0.      0.0001 -0.      0.     -0.     -0.      0.    ]
 [-0.      0.      0.      0.     -0.      0.0003 -0.     -0.     -0.      0.    ]
 [-0.0001 -0.     -0.      0.      0.     -0.      0.0001 -0.      0.     -0.    ]
 [-0.0001  0.     -0.      0.     -0.     -0.     -0.      0.0002  0.0001  0.0001]
 [-0.0001  0.     -0.      0.     -0.     -0.      0.      0.0001  0.0002  0.0001]
 [-0.0001  0.     -0.      0.      0.      0.     -0.      0.0001  0.0001  0.0002]]


In [21]:
np.array([results1.params,results1.HC0_se,results1.params/results1.HC0_se]).T

array([[  1.2014,   0.0328,  36.6527],
       [  0.1012,   0.0019,  53.1431],
       [  0.0296,   0.0013,  22.5391],
       [ -0.0004,   0.    , -16.1615],
       [ -0.1655,   0.0095, -17.4517],
       [ -0.1115,   0.0161,  -6.9333],
       [  0.119 ,   0.0116,  10.2814],
       [ -0.0458,   0.0139,  -3.2931],
       [ -0.0639,   0.0137,  -4.6615],
       [ -0.0066,   0.0145,  -0.4529]])

We see that being female or African American have negative effects on wages.

Now we turn to FGLS estimators, Feasible Generalized Least Squares Estimators.


In [22]:
from scipy.linalg import toeplitz
resid=results1.resid
resid_fit=sm.OLS(resid[1:], sm.add_constant(resid[:-1])).fit()
rho=resid_fit.params[1]
order=toeplitz(range(len(resid)))
sigma = rho**order

In [23]:
gresults=sm.GLS(y,x,sigma=sigma).fit()
np.array([gresults.params,gresults.bse,gresults.params/gresults.bse]).T

array([[  1.2235,   0.033 ,  37.1014],
       [  0.0996,   0.0018,  55.792 ],
       [  0.0296,   0.0013,  22.6168],
       [ -0.0004,   0.    , -16.747 ],
       [ -0.1648,   0.0093, -17.6437],
       [ -0.1136,   0.0174,  -6.5376],
       [  0.1193,   0.0132,   9.0308],
       [ -0.0461,   0.0146,  -3.1536],
       [ -0.0644,   0.0152,  -4.2279],
       [ -0.0072,   0.0156,  -0.4661]])

We see that the FGLS are similar to the White estimates.

In [24]:
np.array([results1.params,gresults.params]).T

array([[ 1.2014,  1.2235],
       [ 0.1012,  0.0996],
       [ 0.0296,  0.0296],
       [-0.0004, -0.0004],
       [-0.1655, -0.1648],
       [-0.1115, -0.1136],
       [ 0.119 ,  0.1193],
       [-0.0458, -0.0461],
       [-0.0639, -0.0644],
       [-0.0066, -0.0072]])

The White estimators are more restrictive than the FGLS estimators.  However it is always good to check for robustness with respect to estimaiton procedure